# WORD GRAPH PROJECT - step 1

We will use the work done here 

https://www.kaggle.com/datasets/moxxis/harry-potter-lstm?resource=download&select=Harry_Potter_all_books_preprocessed.txt

so thank to this boy to provide all the data we searched and to prepocess even them (this should help a lot). 

We start by reading everything.

In [193]:
HP_file = open("HP_books.txt", "r")
HP_text = HP_file.read()

Now we split everything and count how many words there are. We also replace some character with a space (since we had some problem with some sentences) and we lower everything.

In [194]:
special_char = ['.', ',', '?', '!', 'â€˜', 'â€¢', '~', '(', ')']
for s_char in special_char:
    HP_text = HP_text.replace(s_char, " . ")
HP_text = HP_text.lower()
HP_words = HP_text.split(" ")
len(HP_words)

1259690

And we count the numbers of single words.

In [195]:
HP_set_words = set(HP_words)
len(HP_set_words)

23227

This is something really cool to notice. We now there are approximetely 170.000 words in the english vocabulary and this means that HP really minimize the space of the words (relatively talking).
Now we want to count the occurrencies of the single words. 

In [196]:
HP_occourrencies_dict = {}
for word in HP_words:
    if word == '':
        continue
    if word in HP_occourrencies_dict.keys():
        HP_occourrencies_dict[word] += 1
    else:
        HP_occourrencies_dict[word] = 1
HP_sorted_occourrencies_dict = dict(sorted(HP_occourrencies_dict.items(), key=lambda item: item[1], reverse=True))

In [197]:
k = 0
lim = 10
for unique_word in HP_sorted_occourrencies_dict:
    if k < lim:
        print(f"{unique_word} occourred {HP_sorted_occourrencies_dict[unique_word]}")
        k = k + 1

. occourred 86068
the occourred 51921
and occourred 27607
to occourred 26852
of occourred 21843
a occourred 21074
he occourred 20430
harry occourred 16750
was occourred 15644
said occourred 14491


Let's implement a class for the graph.

In [218]:
class WordGraph:
    def __init__(self, text): # we assume the text is already prepocessed
        self.text = text
        self.words = self.text.split(" ")
        self.unique_word = set(self.words)
        self.words_dict = {u_word : {} for u_word in self.unique_word}
        self.compute_graph()
    
    def compute_graph(self):
        for predecessor, successor in zip(self.words[:-1], self.words[1:]):
            if predecessor == '' or successor == '':
                continue
            if successor not in self.words_dict[predecessor].keys():
                self.words_dict[predecessor][successor] = 1
            else:
                self.words_dict[predecessor][successor] += 1
    
    def print_graph(self):
        for words in self.words_dict:
            print(f"{words} ")
            for word in self.words_dict[words]:
                print(f" |->{word} : {self.words_dict[words][word] }")
            print(" ")
    
    def print_word(self, key_word):
        print(f"{key_word} ")
        try:
            w_dict = self.words_dict[key_word]
        except:
            print(" |->Word not found")
            return
        for word in w_dict:
            print(f" |->{word} : {self.words_dict[key_word][word] }")


In [219]:
HP_wg = WordGraph(HP_text)

In [222]:
HP_wg.print_word("voldemort")

voldemort 
 |->was : 68
 |->had : 83
 |->turned : 4
 |->ron : 2
 |->without : 1
 |->could : 9
 |->kill : 1
 |->will : 11
 |->to : 20
 |->certainly : 1
 |->under : 1
 |->killed : 5
 |->on : 2
 |->showed : 2
 |->screamed : 3
 |->again : 3
 |->harry : 5
 |->said : 15
 |->cannot : 2
 |->if : 1
 |->were : 5
 |->whose : 4
 |->himself : 12
 |->might : 3
 |->even : 2
 |->you : 4
 |->heir : 1
 |->got : 3
 |->managed : 1
 |->he : 6
 |->with : 3
 |->who : 13
 |->put : 2
 |->three : 3
 |->can : 4
 |->a : 7
 |->returned : 6
 |->s : 14
 |->and : 55
 |->murdering : 2
 |->when : 1
 |->after : 2
 |->lupin : 1
 |->killing : 1
 |->that : 2
 |->teach : 1
 |->went : 2
 |->met : 1
 |->would : 16
 |->didnt : 2
 |->comes : 1
 |->back : 2
 |->wants : 6
 |->yet : 2
 |->muggle : 1
 |->couldnt : 2
 |->lurking : 1
 |->the : 11
 |->rising : 2
 |->have : 6
 |->is : 14
 |->unless : 1
 |->supporter : 2
 |->disappeared : 1
 |->fell : 2
 |->getting : 1
 |->accusing : 1
 |->professor : 1
 |->looked : 4
 |->slipped : 1
 |